In [10]:
# Turn BioCreative6 TSV format into brat files

In [7]:
import pandas as pd

In [12]:
path_relationships =  '/home/antonis/data/biocreative6/ChemProt_Corpus/chemprot_development/chemprot_development_relations.tsv'


In [13]:
save_path_root = '/home/antonis/data/biocreative6/ChemProt_Corpus/brat/chemprot_development/'

In [14]:
abstracts = pd.read_table(path_abstracts,names=['doc_id', 'title', 'abstract'])

In [15]:
abstracts.title[0]

'Binding of dimemorfan to sigma-1 receptor and its anticonvulsant and locomotor effects in mice, compared with dextromethorphan and dextrorphan.'

In [16]:
len(abstracts)

612

# Parse abstracts, titles

In [4]:
path_abstracts = '/home/antonis/data/biocreative6/ChemProt_Corpus/chemprot_training/chemprot_training_abstracts.tsv'

In [2]:
from preprocessing import BioC_corpus_parse

In [3]:
BioC_corpus_parse(load_path = path_abstracts
                  , save_path = '/home/antonis/data/biocreative6/ChemProt_Corpus/brat/chemprot_training')

Successfully rote 1019 docs to txt (/home/antonis/data/biocreative6/ChemProt_Corpus/brat/chemprot_training)


# Parse entities (into .ann)

In [20]:
path_entities =  '/home/antonis/data/biocreative6/ChemProt_Corpus/chemprot_training/chemprot_training_entities.tsv'
path_relationships = '/home/antonis/data/biocreative6/ChemProt_Corpus/chemprot_training/chemprot_training_relations.tsv'
save_path = '/home/antonis/data/biocreative6/ChemProt_Corpus/brat/chemprot_training'

In [47]:
def BioC_ent_parse(path_entities):
    '''Input: tsv BioCreative format'''
    
    entities = pd.read_table(path_entities,names=['doc_id','nr', 'type', 'start', 'stop', 'name'])
    #hack to write type, start stop without \t
    entities['type_start_stop'] = map(lambda x,y,z: "%s %i %i"%(x,y,z) , entities.type, entities.start, entities.stop)
    return entities

In [48]:
def BioC_rel_parse(path_relationships):
    '''Input: tsv BioCreative format'''
    
    relationships = pd.read_table(path_relationships, names = ['doc_id', 'group', 'eval', 'CPR', 'arg1', 'arg2'])
    relationships['regulator'] = map(lambda x: 'REGULATOR' if x in ['CPR:3', 'CPR:4'] else 'NONREGULATOR'  ,relationships.group)
    print 'Total number of relationships:'
    print relationships['regulator'].value_counts()
    
    # encode in same column
    relationships['rel'] = map(lambda x,y,z: "%s %s %s"%(x,y,z) , relationships.regulator, relationships.arg1, relationships.arg2)
    #enrich with index (eg. R28)
    relationships['idx'] = map(lambda x: "R%i"%x ,relationships.index)
    
    return relationships

In [55]:
def write_ann_file(entities, relationships, save_path, debug=False):
    """Jointly write entities and relationships in .ann files 
    given their dataframes
    
    entities cols:        ['doc_id,'nr','type_start_stop','name']
    relationships cols:   ['doc_id,'nr','idx','rel']
    """
    for doc_id in entities.doc_id.unique():
        
        save_path = save_path.rstrip("/")#trim trailing /
        save_path_file = '%s/%s.ann'%(save_path,doc_id)

        #write out entities
        entities[entities.doc_id==doc_id][['nr','type_start_stop','name']].to_csv(save_path_file,sep='\t',header=None,index=False)
        #write out relationships
        relationships[relationships.doc_id==doc_id][['idx', 'rel']].to_csv(save_path_file,sep='\t',header=None,mode='a',index=False)

        if debug:
            print save_path_file.split('/')[-1], ': ',len(entities[entities.doc_id==doc_id]), 'entities', len(relationships[relationships.doc_id==doc_id]),'relations'
    print "Successfully wrote %i files in: %s" %( len(entities.doc_id.unique()), 
                                                 save_path)

In [57]:
write_ann_file(entities, relationships, save_path)

Successfully wrote 1020 files in: /home/antonis/data/biocreative6/ChemProt_Corpus/brat/chemprot_training


In [13]:
entities = pd.read_table(path_entities,names=['doc_id','nr', 'type', 'start', 'stop', 'name'])

In [14]:
#hack to write type, start stop without \t
entities['type_start_stop'] = map(lambda x,y,z: "%s %i %i"%(x,y,z) , entities.type, entities.start, entities.stop)

In [15]:
# entities = entities[['doc_id','nr','type_start_stop','name']]

In [16]:
entities.head()

,doc_id,nr,type,start,stop,name,type_start_stop
0,11319232,T1,CHEMICAL,242,251,acyl-CoAs,CHEMICAL 242 251
1,11319232,T2,CHEMICAL,1193,1201,triacsin,CHEMICAL 1193 1201
2,11319232,T3,CHEMICAL,1441,1448,sucrose,CHEMICAL 1441 1448
3,11319232,T4,CHEMICAL,1637,1652,triacylglycerol,CHEMICAL 1637 1652
4,11319232,T5,CHEMICAL,1702,1711,acyl-CoAs,CHEMICAL 1702 1711


# Parse relationships


In [21]:
relationships = pd.read_table(path_relationships, names = ['doc_id', 'group', 'eval', 'CPR', 'arg1', 'arg2'])

In [22]:
relationships.head()

,doc_id,group,eval,CPR,arg1,arg2
0,10047461,CPR:3,Y,ACTIVATOR,Arg1:T13,Arg2:T57
1,10047461,CPR:3,Y,ACTIVATOR,Arg1:T7,Arg2:T39
2,10047461,CPR:3,Y,ACTIVATOR,Arg1:T7,Arg2:T40
3,10047461,CPR:3,Y,ACTIVATOR,Arg1:T7,Arg2:T41
4,10047461,CPR:3,Y,INDIRECT-UPREGULATOR,Arg1:T13,Arg2:T55


In [28]:
relationships['regulator'] = map(lambda x: 'REGULATOR' if x in ['CPR:3', 'CPR:4'] else 'NONREGULATOR'  ,relationships.group)

In [27]:
print 'Number of relationships:'
print relationships['regulator'].value_counts()

Number of relationships:
OTHER        3400
REGULATES    3037
Name: regulator, dtype: int64


In [31]:
# encode in same column
relationships['rel'] = map(lambda x,y,z: "%s %s %s"%(x,y,z) , relationships.regulator, relationships.arg1, relationships.arg2)
#enrich with index (eg. R28)
relationships['idx'] = map(lambda x: "R%i"%x ,relationships.index)

In [46]:
# jointly write entities and relationships in .ann files
for doc_id in entities.doc_id.unique():
    save_path_file = '%s/%s.ann'%(save_path,doc_id)

    #write out entities
    entities[entities.doc_id==doc_id][['nr','type_start_stop','name']].to_csv(save_path_file,sep='\t',header=None,index=False)
    
    #write out relationships
    relationships[relationships.doc_id==doc_id][['idx', 'rel']].to_csv(save_path_file,sep='\t',header=None,mode='a',index=False)
    
    print save_path_file.split('/')[-1], ': ',len(entities[entities.doc_id==doc_id]), 'entities', len(relationships[relationships.doc_id==doc_id]),'relations'


11319232.ann :  38 entities 12 relations
12213829.ann :  55 entities 14 relations
19228178.ann :  17 entities 6 relations
15843826.ann :  20 entities 4 relations
23628509.ann :  28 entities 10 relations
23375813.ann :  10 entities 3 relations
23538162.ann :  31 entities 16 relations
21272013.ann :  16 entities 10 relations
23142507.ann :  26 entities 8 relations
22989705.ann :  32 entities 9 relations
16686685.ann :  25 entities 3 relations
22972179.ann :  30 entities 8 relations
23570914.ann :  23 entities 6 relations
15953344.ann :  23 entities 9 relations
1386491.ann :  17 entities 5 relations
1975995.ann :  9 entities 1 relations
23238103.ann :  26 entities 4 relations
23357629.ann :  20 entities 3 relations
10207608.ann :  31 entities 31 relations
9079657.ann :  45 entities 2 relations
18559268.ann :  13 entities 6 relations
2886642.ann :  29 entities 2 relations
12560076.ann :  28 entities 6 relations
23500776.ann :  22 entities 4 relations
23232123.ann :  11 entities 0 relations

1375507.ann :  34 entities 16 relations
8263811.ann :  44 entities 11 relations
22659088.ann :  41 entities 2 relations
11034389.ann :  33 entities 3 relations
11156594.ann :  35 entities 8 relations
9928259.ann :  26 entities 5 relations
23192350.ann :  40 entities 1 relations
20230007.ann :  46 entities 20 relations
1332478.ann :  13 entities 1 relations
23409864.ann :  7 entities 0 relations
23537942.ann :  8 entities 0 relations
23561109.ann :  8 entities 0 relations
23266719.ann :  23 entities 0 relations
17275903.ann :  37 entities 2 relations
23454300.ann :  6 entities 0 relations
3828735.ann :  24 entities 4 relations
19549602.ann :  30 entities 16 relations
19203467.ann :  81 entities 26 relations
12955294.ann :  19 entities 0 relations
17111172.ann :  37 entities 15 relations
23046524.ann :  16 entities 7 relations
14642426.ann :  20 entities 11 relations
23422569.ann :  53 entities 17 relations
15795320.ann :  42 entities 10 relations
15561973.ann :  42 entities 2 relations


23517912.ann :  44 entities 23 relations
23535394.ann :  12 entities 2 relations
23403203.ann :  29 entities 10 relations
23305850.ann :  34 entities 16 relations
23471540.ann :  18 entities 0 relations
17081103.ann :  39 entities 1 relations
10444166.ann :  19 entities 3 relations
23542147.ann :  35 entities 1 relations
8799556.ann :  60 entities 28 relations
19370562.ann :  20 entities 2 relations
19389876.ann :  20 entities 4 relations
23519392.ann :  23 entities 7 relations
16441928.ann :  40 entities 0 relations
23493374.ann :  54 entities 11 relations
23291240.ann :  28 entities 0 relations
17192395.ann :  19 entities 2 relations
10617684.ann :  31 entities 11 relations
23249624.ann :  36 entities 8 relations
1977506.ann :  31 entities 7 relations
23122096.ann :  10 entities 0 relations
16480505.ann :  15 entities 11 relations
23023136.ann :  32 entities 4 relations
17035141.ann :  41 entities 0 relations
12724156.ann :  43 entities 6 relations
17549299.ann :  14 entities 0 relat

23536521.ann :  23 entities 1 relations
12387707.ann :  23 entities 2 relations
16597615.ann :  17 entities 0 relations
11511086.ann :  30 entities 16 relations
17308037.ann :  59 entities 29 relations
14871882.ann :  48 entities 0 relations
16291873.ann :  22 entities 10 relations
23177256.ann :  19 entities 15 relations
9669506.ann :  42 entities 6 relations
10826417.ann :  8 entities 2 relations
23560442.ann :  2 entities 0 relations
23357976.ann :  22 entities 5 relations
17352516.ann :  30 entities 14 relations
23500545.ann :  23 entities 5 relations
12881227.ann :  43 entities 16 relations
23333261.ann :  21 entities 5 relations
12517247.ann :  33 entities 4 relations
23397052.ann :  19 entities 0 relations
16303771.ann :  42 entities 6 relations
23481236.ann :  53 entities 29 relations
16702329.ann :  34 entities 8 relations
8242725.ann :  17 entities 6 relations
23142538.ann :  19 entities 5 relations
16547651.ann :  37 entities 4 relations
22902307.ann :  2 entities 0 relation

23614367.ann :  20 entities 0 relations
23429912.ann :  40 entities 19 relations
15689518.ann :  20 entities 9 relations
2083404.ann :  30 entities 9 relations
15133083.ann :  31 entities 3 relations
10446934.ann :  51 entities 12 relations
3299141.ann :  36 entities 0 relations
23411259.ann :  13 entities 0 relations
11009561.ann :  37 entities 16 relations
17334708.ann :  14 entities 2 relations
12606421.ann :  56 entities 2 relations
15121646.ann :  34 entities 12 relations
15114505.ann :  28 entities 18 relations
23063069.ann :  13 entities 0 relations
19708854.ann :  11 entities 0 relations
23567953.ann :  5 entities 0 relations
22705396.ann :  5 entities 0 relations
23514378.ann :  11 entities 1 relations
23473032.ann :  27 entities 1 relations
15908512.ann :  31 entities 16 relations
17616291.ann :  24 entities 0 relations
12067524.ann :  30 entities 11 relations
23265892.ann :  10 entities 3 relations
23291559.ann :  27 entities 7 relations
23318726.ann :  34 entities 12 relati

In [43]:
save_path

'/home/antonis/data/biocreative6/ChemProt_Corpus/brat/chemprot_training'

### # # # # # # 
### OLD CODE

In [24]:
#filter out rels not related in regulation
relationships_regul = relationships[(relationships['group']=='CPR:3') | (relationships['group']=='CPR:4')]

In [40]:
print "Total + relationships: %i (%.2f%%)" %(len(relationships_regul), float(len(relationships_regul))/len(relationships))
print "Total # entities:", len(entities)
print "Total documents:", len(entities.doc_id.unique())

Total + relationships: 1655 (0.47%)
Total # entities: 15567
Total documents: 612


In [123]:
# hack to write without \t
relationships_regul['rel'] = map(lambda x,y,z: "%s %s %s"%(x,y,z) , relationships_regul.CPR, relationships_regul.arg1, relationships_regul.arg2)

/home/antonis/anaconda2/envs/python27/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [142]:
relationships_regul['idx'] = map(lambda x: "R%i"%x ,relationships.index)

In [144]:
relationships_regul = relationships_regul[['doc_id','idx','rel']]

In [145]:
relationships_regul.tail()

,doc_id,idx,rel
3553,9876137,R3553,INHIBITOR Arg1:T5 Arg2:T13
3554,9876137,R3554,INHIBITOR Arg1:T5 Arg2:T14
3555,9876137,R3555,INHIBITOR Arg1:T5 Arg2:T16
3556,9876137,R3556,INHIBITOR Arg1:T6 Arg2:T16
3557,9876137,R3557,INHIBITOR Arg1:T8 Arg2:T23


** How many relationships in how many abstracts? **

In [49]:
print( "Docs with entities: %s "%len(entities.doc_id.unique()))
print "Docs with any kind of relations: %i (%.2f%%)"%(len(relationships.doc_id.unique()), len(relationships.doc_id.unique())/float(len(entities.doc_id.unique())))
print "Docs with + relations: %i (%.2f%%)"%(len(relationships_regul.doc_id.unique()), len(relationships_regul.doc_id.unique())/float(len(entities.doc_id.unique())))


Docs with entities: 612 
Docs with any kind of relations: 443 (0.72%)
Docs with + relations: 294 (0.48%)


# Write file out

In [146]:
for doc_id in entities.doc_id.unique():
    save_path = '%s%s.ann'%(save_path_root,doc_id)

    #write out entities
    entities[entities.doc_id==doc_id].drop('doc_id',axis=1).to_csv(save_path,sep='\t',header=None,index=False)
    
    #write out relationships
    relationships_regul[relationships_regul.doc_id==doc_id].drop('doc_id',axis=1).to_csv(save_path,sep='\t',header=None,mode='a',index=False)
    
    print save_path.split('/')[-1], ': ',len(entities[entities.doc_id==doc_id]), 'entities', len(relationships_regul[relationships_regul.doc_id==doc_id]),'relations'


10064839.ann :  56 entities 0 relations
10082498.ann :  34 entities 5 relations
10193663.ann :  35 entities 0 relations
10193665.ann :  30 entities 0 relations
10226872.ann :  31 entities 0 relations
10362188.ann :  25 entities 2 relations
10373224.ann :  35 entities 9 relations
10381787.ann :  22 entities 8 relations
10432475.ann :  62 entities 0 relations
10439935.ann :  28 entities 2 relations
10460702.ann :  44 entities 3 relations
10531013.ann :  14 entities 8 relations
10546982.ann :  20 entities 3 relations
10575283.ann :  18 entities 0 relations
10579749.ann :  45 entities 0 relations
10637238.ann :  18 entities 0 relations
10683249.ann :  36 entities 1 relations
10688973.ann :  41 entities 0 relations
10697523.ann :  24 entities 5 relations
10698696.ann :  30 entities 0 relations
10742288.ann :  25 entities 0 relations
10796070.ann :  19 entities 1 relations
10800083.ann :  34 entities 0 relations
10839333.ann :  21 entities 1 relations
10866053.ann :  33 entities 0 relations


17627474.ann :  32 entities 2 relations
17632548.ann :  50 entities 0 relations
17704420.ann :  33 entities 0 relations
17768678.ann :  44 entities 11 relations
17869551.ann :  37 entities 2 relations
17916059.ann :  16 entities 0 relations
17980251.ann :  30 entities 0 relations
18045203.ann :  15 entities 0 relations
18450949.ann :  20 entities 4 relations
18466433.ann :  3 entities 0 relations
18480678.ann :  32 entities 2 relations
18593822.ann :  64 entities 0 relations
18593927.ann :  15 entities 1 relations
18604229.ann :  29 entities 2 relations
18620538.ann :  14 entities 0 relations
18655903.ann :  80 entities 0 relations
1868500.ann :  29 entities 13 relations
18803986.ann :  53 entities 10 relations
1881457.ann :  28 entities 6 relations
18973551.ann :  38 entities 6 relations
18974361.ann :  28 entities 4 relations
19002123.ann :  24 entities 0 relations
19023039.ann :  40 entities 13 relations
19057128.ann :  29 entities 13 relations
19119014.ann :  37 entities 16 relatio

23461841.ann :  20 entities 0 relations
23466460.ann :  13 entities 0 relations
23466488.ann :  58 entities 5 relations
23466526.ann :  18 entities 0 relations
23470280.ann :  26 entities 0 relations
23470776.ann :  9 entities 0 relations
23473037.ann :  17 entities 0 relations
23473679.ann :  11 entities 0 relations
23473801.ann :  35 entities 13 relations
23474014.ann :  7 entities 0 relations
23474386.ann :  11 entities 1 relations
23477419.ann :  21 entities 0 relations
23478260.ann :  14 entities 0 relations
23485402.ann :  33 entities 0 relations
23493372.ann :  30 entities 0 relations
23493568.ann :  34 entities 0 relations
23494106.ann :  27 entities 6 relations
23494802.ann :  16 entities 0 relations
23494944.ann :  21 entities 7 relations
23495999.ann :  22 entities 0 relations
23497878.ann :  8 entities 0 relations
23499825.ann :  6 entities 0 relations
23500058.ann :  18 entities 0 relations
23500387.ann :  21 entities 5 relations
23500491.ann :  24 entities 0 relations
235